# A basic model to try and predict the Total Return of Eligible Stocks:
### Predicts the P/E multiple and hence the forecast price -> total return of a group of stocks

## Set-Up

### Import Packages

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import root
import matplotlib.pyplot as plt

### Set Starting Parameters

In [2]:
R = 0.055
b = 1/(1+R)
maturity_g = 0.073
time_frame = 10
start_earnings = 10


## Import the Bloomberg data from Excel

### Read In The Data

In [3]:
asset_df = pd.read_excel("../../data/Bloomberg_Rankings.xlsx", sheet_name = "Mid Cap and Above")

### Inspect the Data

In [13]:
asset_df.head(8)

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple),5Y Multiple,5Y Expected Annualised TR (Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563,79.494055,2.301199
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056,79.494055,2.944991
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944,79.494055,1.275612
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355,79.494055,2.322208
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109,79.494055,2.820447
7,6920 JP Equity,Lasertec Corp,40100.000000,21.450892,0.679599,742.560000,321.568675,8.019169e-03,105.626938,71.924868,71.924868,54.047060,0.018502,11153.856393,0.278151,0.751059,0.923087,79.494055,2.002589
8,KSPI KZ Equity,Kaspi.KZ JSC,56200.000000,19.993069,0.660910,4675.741863,4503.032801,8.012514e-02,36.347841,44.302241,44.302241,12.138770,0.082381,29256.064283,0.520571,0.561900,0.577625,79.494055,6.139020
9,NVDA US Equity,NVIDIA Corp,1131.235840,16.232818,0.607161,17.270000,15.778580,1.394809e-02,794.818653,176.175527,176.175527,65.843123,0.015188,2774.714443,2.452817,1.803700,2.394154,79.494055,1.787941


### Drop the rows where "5Y Forecast EPS Growth" is Null
- We need this for our prediction of the P/E multiple, and then the EPS with which we multiply that multiple by to get the future price. 
- The current rudimentary form of predicting the multiple assumes that investors presume that EPS growth continues at the current rate for a certain set of years and then trails back down to a maturity growth rate. Investors purchase up until the point at which the multiple that they have priced in yields them the minimum expected rate of return, `R`, that they will take.
- I.e., the multiple today depends on the price and hence multiple tomorrow and so on so forth in a geometric series.
- Investors could calculate this `R` in any way, whether it be the CAPM or the Fama-French Three-Factor or another model to identify the best hurdle rate/ cost of equity capital they shoud take

In [14]:
asset_df = asset_df.dropna(subset=['5 YR Forecast EPS Growth'])
# Inspect the column (These figures will be in percentage, not proportion scale)
asset_df["5 YR Forecast EPS Growth"]


0       336.721739
1        92.741867
4        52.594889
5        80.841010
6        63.130366
           ...    
1033     12.294384
1039    -10.572361
1042      8.237525
1045     57.864844
1047     25.676972
Name: 5 YR Forecast EPS Growth, Length: 709, dtype: float64

In [7]:

def predict_multiple(growth, t): 
    growth = min([growth/100,0.30])
    maturity_g = 0.065 + 0.1*(growth - 0.065)
    k_vec_m = 1 + maturity_g
    maturity_multiple = (k_vec_m*b*(1-(k_vec_m*b)**18)/(1-b*k_vec_m))
    maturity_multiple   
    b_vec = [b] * t
    k_vec = [1 + growth] * t
    b_vec = np.cumprod(b_vec)[:t]
    k_vec = np.cumprod(k_vec)[:t]
   
    return sum(k_vec*b_vec) + maturity_multiple*b_vec[-1]*k_vec[-1]


predict_multiple(341.92,5)

79.49405534513394

In [8]:
asset_df["5Y Multiple"] = asset_df["5 YR Forecast EPS Growth"].apply(lambda x: predict_multiple(x, 5))


In [9]:
def predict_roi(growth, multiple, fut_multiple): 
    growth = min([growth/100,0.30])
    return (((1+growth)**6)*(1+(1/multiple)+(fut_multiple/multiple)))**1/6


## Predict the ROI Corresponding to this

In [10]:

asset_df["5Y Expected Annualised TR (Change in Multiple)"] = [predict_roi(asset_df["5 YR Forecast EPS Growth"][i], asset_df["Multiple"][i], asset_df["5Y Multiple"][i]) for i in asset_df.index]

In [11]:
asset_df.head()

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple),5Y Multiple,5Y Expected Annualised TR (Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563,79.494055,2.301199
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056,79.494055,2.944991
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944,79.494055,1.275612
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355,79.494055,2.322208
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109,79.494055,2.820447


In [12]:
asset_df.to_excel("../../data/Top_Stocks.xlsx")